## Workspace Management demo

This notebook presents examples of how to manage user workspaces.

In [ ]:
import requests
import datetime

In [ ]:
base_domain = "demo.eoepca.org"
# base_domain = "develop.eoepca.org"
#workspaces_endpoint = f'https://workspace-api-open.{base_domain}/workspaces'
workspaces_endpoint = f'https://workspace-api.{base_domain}/workspaces'

We want to associate the workspace with a user. Also, some operations on the workspace require a logged in user. So we are using the client and use Eric as an owner.

In [ ]:
import utils.DemoClient as client
import jwt

demo = client.DemoClient(f"https://auth.{base_domain}")
#demo = client.DemoClient(f"https://auth.{base_domain}")
demo.register_client()
demo.save_state()

In [ ]:
USER_ERIC="eric"
USER_ERIC_PASSWORD="defaultPWD"
eric_id_token = demo.get_id_token(USER_ERIC, USER_ERIC_PASSWORD)
headers = {
    'Authorization': 'Bearer ' + eric_id_token
}

jwt_header = jwt.get_unverified_header(eric_id_token)
jwt_payload = jwt.decode(eric_id_token, options={"verify_signature": False})
eric_uid = jwt_payload["sub"]

In [ ]:
USER_ADMIN="admin"
USER_ADMIN_PASSWORD="admin_Abcd1234#"
admin_id_token = demo.get_id_token(USER_ADMIN, USER_ADMIN_PASSWORD)
headers = {
    'Authorization': 'Bearer ' + admin_id_token
}

jwt_header = jwt.get_unverified_header(admin_id_token)
jwt_payload = jwt.decode(admin_id_token, options={"verify_signature": False})
admin_uid = jwt_payload["sub"]

### Creation and retrieval of credentials

The following request will create a user workspace.

In [ ]:
preferred_workspace_name = f"my-new-user-{datetime.datetime.now().timestamp():.0f}"
preferred_workspace_name

In [ ]:
response = requests.post(
    workspaces_endpoint,
    headers=headers,
    json={
        "preferred_name": preferred_workspace_name,
        "default_owner": admin_uid,
    }
)
response.raise_for_status()
actual_workspace_name = response.json()['name']
response.json()

Workspace creation is an asynchronous process, so when we try to retrieve the workspace data right now, we see that it is still provisioning.

In [ ]:
response = requests.get(f"{workspaces_endpoint}/{actual_workspace_name}", headers=headers)
response.raise_for_status()
response.json()

In [ ]:
import time
# wait for provisioning
time.sleep(10)

Now all the credentials should be set up and are available:

In [ ]:
response = requests.get(f"{workspaces_endpoint}/{actual_workspace_name}", headers=headers)
response.raise_for_status()

workspace_data = response.json()
bucket_name = workspace_data["storage"]["credentials"]["bucketname"]
s3_access = workspace_data["storage"]["credentials"]["access"]
s3_secret = workspace_data["storage"]["credentials"]["secret"]
workspace_data["storage"]["credentials"]['secret'] = "hidden_for_demo"
workspace_data["container_registry"]["password"] = "hidden_for_demo"
workspace_data

## Registration

We will now register an ADES instance in the workspace API

In [ ]:
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/register",
    headers=headers,
    json={
        "type": "ades",
        "url": "https://demo.pygeoapi.io/stable/processes",
    }
)
response.raise_for_status()
response

Check if the ADES is actually registered:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{actual_workspace_name}.{base_domain}/collections/metadata:main/items?type=service", headers=dict(f="json", **headers))
response.raise_for_status()
response.json()

We will now register an OGC API - Processes instance in the workspace API:

In [ ]:
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/register",
    headers=headers,
    json={
        "type": "oaproc",
        "url": "https://demo.pygeoapi.io/stable",
    }
)
response.raise_for_status()
response

Check if the OGC API - Processes is actually registered:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{actual_workspace_name}.{base_domain}/collections/metadata:main/items?type=service", headers=dict(f="json", **headers))
response.raise_for_status()
response.json()

We will now register an OGC API - Records instance in the workspace API:

In [ ]:
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/register",
    headers=headers,
    json={
        "type": "catalogue",
        "url": "http://demo.pycsw.org/gisdata",
    }
)
response.raise_for_status()
response

Check if the OGC API - Records is actually registered:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{actual_workspace_name}.{base_domain}/collections/metadata:main/items?type=service", headers=dict(f="json", **headers))
response.raise_for_status()
response.json()

We will now register an OGC CSW instance in the workspace API:

In [ ]:
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/register",
    headers=headers,
    json={
        "type": "catalogue",
        "url": "http://demo.pycsw.org/gisdata/csw",
    }
)
response.raise_for_status()
response

Check if the OGC CSW catalogue is actually registered:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{actual_workspace_name}.{base_domain}/collections/metadata:main/items?type=service", headers=dict(f="json", **headers))
response.raise_for_status()
response.json()

Now we create a new collection:

In [ ]:
collection = requests.get("https://raw.githubusercontent.com/radiantearth/stac-spec/master/examples/collection.json").json()

response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/register-collection",
    headers=headers,
    json=collection, # TODO
)
response.raise_for_status()
response

To see if it was registered:

In [ ]:
time.sleep(3)
response = requests.get(f"http://resource-catalogue.{actual_workspace_name}.{base_domain}/collections/metadata:main/items/simple-collection", headers=headers)
response.raise_for_status()
response.json()

We will now upload an application package to the workspace:

In [ ]:
import boto3
#S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
S3_ENDPOINT = "https://minio.demo.eoepca.org"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)
object = s3resource.Object(bucket_name, 'application-package/s-expression/s-expression-0_0_2.cwl')
result = object.put(Body=open('../data/s-expression-cwl.cwl', 'rb'))
res = result.get('ResponseMetadata')
if res.get('HTTPStatusCode') == 200:
    print('Application package uploaded successfully')
else:
    print('Application package not uploaded')

Now we register the application package:

In [ ]:
resource_url= f'{bucket_name}/application-package/s-expression/s-expression-0_0_2.cwl'
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/register",
    headers=headers,
    json={
        "type": "application",
        "url": resource_url,
    }
)
response.raise_for_status()
response

Now we check the registration result:

In [ ]:
time.sleep(3)
response = requests.get(f"https://resource-catalogue.{actual_workspace_name}.{base_domain}/collections/metadata:main/items?type=application", headers=dict(f="json", **headers))
response.raise_for_status()
response.json()

## Redeploy

In case something went wrong or an update is necessary, it's possible to trigger the provisioning again:

In [ ]:
response = requests.post(
    f"{workspaces_endpoint}/{actual_workspace_name}/redeploy",
    headers=headers
)
response.raise_for_status()
response

## Deletion
Workspaces can also be deleted like this:

In [ ]:
response = requests.delete(
    f"{workspaces_endpoint}/{actual_workspace_name}",
    headers=headers
)
response.raise_for_status()
response